In [0]:
%pip install lightning

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
%run
./display_images

#Imports

In [0]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import pickle
import datetime
import pytz
from numpy.random import choice
from google.cloud import storage
import io
import lightning.pytorch as pl
import torch.nn.functional as F


storage_client = storage.Client()
bucket = storage_client.get_bucket('gcs-dsci-fryou-fy-dev-prd')

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7a11fc267e20>
Traceback (most recent call last):
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/databricks/python/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
2025-03-12 06:14:55.733413: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU in

#Functions

In [0]:
def get_index(index_path, entity, special_tokens, shift_index=True):
    bucket.blob(blob_name=f"{index_path}{entity}_index.pkl").download_to_filename(
        f"{entity}_index.pkl"
    )
    with open(f"{entity}_index.pkl", "rb") as file:
        entity_index = pickle.load(file)
    if shift_index:
        special_tokens_len = len(special_tokens)
        entity_index = special_tokens | {k:v+special_tokens_len for k,v in entity_index.items()}
    return entity_index, len(entity_index)

In [0]:
def get_batch_data(interaction_seq):

    end_time = interaction_seq[-1]['event_time']
    start_time = end_time - timedelta(days=future_window)

    input_seq = [event for event in interaction_seq if event['event_time'] < start_time ]
    input_seq = input_seq[-1 * int(behaviour_context_length):]

    positive_seq = [event for event in interaction_seq if event['event_time'] >= start_time ]
    print(len(positive_seq), len(input_seq), len(interaction_seq))

    catalog_id_seq = np.array([catalog_id_index.get(event['catalog_id'], catalog_id_index['<MASK>']) for event in input_seq])
    scat_id_seq = np.array([sscat_id_index.get(event['sscat_id'], sscat_id_index['<MASK>']) for event in input_seq])
    event_type_seq = np.array([event_type_index[event['event_type']] for event in input_seq])
    event_time_seq = np.array([event['event_time'] for event in input_seq])

    seq_len = len(catalog_id_seq)

    positive_catalog_id_seq = np.array([catalog_id_index.get(event['catalog_id'], catalog_id_index['<MASK>']) for event in positive_seq])
    positive_sscat_id_seq = np.array([sscat_id_index.get(event['sscat_id'], sscat_id_index['<MASK>']) for event in positive_seq])

    if seq_len < behaviour_context_length:
        pad_num = behaviour_context_length - seq_len
        catalog_id_seq = np.pad(catalog_id_seq, (pad_num, 0))
        scat_id_seq = np.pad(scat_id_seq, (pad_num, 0))
        event_type_seq = np.pad(event_type_seq, (pad_num, 0))
        event_time_seq = np.pad(event_time_seq, (pad_num, 0))

    X = {
        "behaviour_seq": torch.tensor([catalog_id_seq]).long(),
        "sscat_id_seq": torch.tensor([scat_id_seq]).long(),
        "event_type_seq": torch.tensor([event_type_seq]).long(),
        "event_time_seq": event_time_seq,

        "positive_catalog_seq": torch.tensor([positive_catalog_id_seq]).long(),
        "positive_sscat_id_seq": torch.tensor([positive_sscat_id_seq]).long(),
    }

    return X



In [0]:
import torch.nn.functional as F


def get_recos(interaction_list, k=5):
    X = get_batch_data(interaction_list)
    X['selection_indices']=torch.tensor([-1]).long()
    user_e, sscat_logits = user_tower(X, catalog_embedding)
    sscats = [index_to_sscat[ele] for ele in sscat_logits.topk(k)[1][0][0].tolist()]
    return X, sscats, sscat_logits

# schema = T.StructType([
#     T.StructField("st", T.ArrayType(T.IntegerType())),
#     T.StructField("mt", T.ArrayType(T.IntegerType())),
#     T.StructField("lt", T.ArrayType(T.IntegerType()))
# ])

# get_recos_udf = F.udf(get_recos, schema)


#Model

In [0]:
class CatalogEmbedding(nn.Module):
    def __init__(
        self, 
        catalog_id_vocab_size,
        embedding_dim,
        use_pretrained=False,
        pretrained_weights=None,
    ):
        super().__init__()

        self.catalog_id_vocab_size = catalog_id_vocab_size
        self.embedding_dimmension = embedding_dim

        self.catalog_embedding = nn.Embedding(num_embeddings=catalog_id_vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        if use_pretrained:
            self.catalog_embedding.weight = nn.Parameter(pretrained_weights, requires_grad=False)

    def forward(self, X):
        x = self.catalog_embedding(X)
        return x

In [0]:
class UserTower(nn.Module):
    def __init__(self,
                 num_layers,
                 nhead,
                 behaviour_context_length,
                 embedding_dim,
                 event_type_vocab_size,
                 sscat_vocab_size,
                 batch_size
                ):
        super(UserTower, self).__init__()

        self.nhead = nhead
        dropout_perc = 0.3 
        self.behaviour_context_length = behaviour_context_length
        self.embedding_dim = embedding_dim
        self.sscat_vocab_size = sscat_vocab_size

        # Event type embedding
        self.event_type_embedding = nn.Embedding(event_type_vocab_size, embedding_dim)
        num_features = 2
        time_features = 0
        self.input_dim = num_features * embedding_dim + time_features

        # Transformer layers
        self.dropout = nn.Dropout(dropout_perc)
        self.norm = nn.LayerNorm(embedding_dim)
        encoder_layer = nn.TransformerEncoderLayer(embedding_dim, nhead=nhead, batch_first=True, dropout=dropout_perc, norm_first=True)
        self.tf_encoder = nn.TransformerEncoder(encoder_layer, num_layers, norm=self.norm)

        # MLP head
        self.mlp_extract = nn.Linear(self.input_dim, embedding_dim)

        self.mlp_out = nn.Sequential(
            nn.LayerNorm(embedding_dim),
            nn.Linear(embedding_dim, 4 * embedding_dim),
            nn.GELU(),
            nn.Linear(4 * embedding_dim, embedding_dim),
            nn.LayerNorm(embedding_dim)
        )

        self.sscat_mlp = nn.Linear(embedding_dim, self.sscat_vocab_size)

        self.future_mask = torch.triu(torch.ones(self.behaviour_context_length, self.behaviour_context_length, dtype=torch.bool), diagonal=1)
        self.register_buffer('future_mask_const', self.future_mask)
        self.register_buffer('seq_diag_const', ~torch.diag(torch.ones(self.behaviour_context_length, dtype=torch.bool)))
        self.register_buffer('batch_indices', torch.arange(batch_size))

    def merge_attn_masks(self, X):
        batch_size = X["behaviour_seq"].shape[0]
        seq_len = X["behaviour_seq"].shape[1]

        padding = (X["behaviour_seq"] == 0)
        masking = (X["behaviour_seq"] == 1)
        padding_mask = padding.logical_or(masking)

        padding_mask_broadcast = padding_mask.bool().unsqueeze(1)
        future_masks = torch.tile(self.future_mask_const[:seq_len, :seq_len], (batch_size, 1, 1))

        merged_masks = torch.logical_or(padding_mask_broadcast, future_masks)
        
        diag_masks = torch.tile(self.seq_diag_const[:seq_len, :seq_len], (batch_size * self.nhead, 1, 1))
        multi_head_mask = torch.tile(merged_masks.unsqueeze(1), (1, self.nhead, 1, 1)).reshape((-1, seq_len, seq_len))
        
        return torch.logical_and(diag_masks, multi_head_mask)

    def positional_encoding(self, timestamps, embedding_dim=16):
        pe = torch.stack((torch.sin(timestamps.unsqueeze(-1) * self.div_term), torch.cos(timestamps.unsqueeze(-1) * self.div_term)), dim=-1).flatten(start_dim=-2)
        return pe

    def forward(self, X, catalog_embedding):
        # print("Input X:", X)
        # print("Catalog Embedding:", catalog_embedding)

        x = torch.cat((
            catalog_embedding(X['behaviour_seq']), 
            self.event_type_embedding(X['event_type_seq'])
        ), dim=-1)
        # print("Concatenated Embedding:", x)

        x = self.mlp_extract(x)
        # print("MLP Extract Output:", x)

        attn_mask = self.merge_attn_masks(X)
        # print("Attention Mask:", attn_mask)

        x = self.tf_encoder(x, mask=attn_mask, is_causal=True)
        # print("Transformer Encoder Output:", x)

        batch_indices = self.batch_indices[:X['selection_indices'].shape[0]].view(-1, 1)
        x = x[batch_indices, X['selection_indices'], :]
        # print("Selected Embeddings:", x)

        x = self.mlp_out(x)
        # print("MLP Out Output:", x)

        x = F.normalize(x, p=2, dim=-1)
        # print("Normalized Embedding:", x)

        sscat_logits = self.sscat_mlp(x)
        # print("SSCat Logits:", sscat_logits)

        return x, sscat_logits
    

In [0]:
class CatalogTower(nn.Module):
    def __init__(self, embedding_dim):
        super(CatalogTower, self).__init__()

        self.mlp = nn.Sequential(
            nn.LayerNorm(embedding_dim),
            nn.Linear(embedding_dim, 4 * embedding_dim),
            nn.GELU(),
            nn.Linear(4 * embedding_dim, embedding_dim),
            nn.LayerNorm(embedding_dim)
        )
    
    def forward(self, x, catalog_embedding):
        x = catalog_embedding(x)
        x = self.mlp(x)
        x = F.normalize(x, p=2, dim=-1)
        return x

#Config

In [0]:
config = {
    # data params 
    "checkpoint_version": "v15.1",
    "index_path": "long_term_user_emb/indexes/v15/",
    "filtered_journeys_path": "long_term_user_emb/user_seq_final/v15/",
    "checkpoint_path": "gs://gcs-dsci-fryou-fy-dev-prd/long_term_user_emb/checkpoint_dir/v15.1/",

    # dataset params
    "behaviour_context_length": 1000,
    "all_dense_perc": 0.15,
    "future_window": 14,
    "max_positives": 64,
    "negatives": 10,
    "batch_size": 512,
    "num_workers": 8,

    # model params
    "embedding_dim": 16,
    "num_layers": 2,
    "nhead": 2,

    # training params 
    "max_epochs": 2,
    "limit_val_batches": 500,
    "limit_train_batches": 20000,
    "val_check_interval": 500,
    "overfit_batches": 0,
    "log_every_n_steps": 200,
    "accelerator": "gpu",
    "num_gpu": 1,
    "num_nodes": 1,
    "num_sanity_val_steps": 2
}

In [0]:
future_window = config['future_window']
behaviour_context_length = config['behaviour_context_length']

#Path

In [0]:
start_date = '2024-01-01'
end_date = '2024-12-32'


#Indexes

In [0]:
pl.seed_everything(42, workers=True)
SPECIAl_TOKENS = {
    "<PAD>":0,
    "<MASK>":1
}

event_type_index, event_type_vocab_size = get_index(config["index_path"], "event_type", SPECIAl_TOKENS)
catalog_id_index, catalog_id_vocab_size = get_index(config["index_path"], "catalog_id", SPECIAl_TOKENS)
sscat_id_index, sscat_id_vocab_size = get_index(config["index_path"], "sscat_id", SPECIAl_TOKENS)

print(f"catalog vocab size:{catalog_id_vocab_size/1000000} M")


INFO: [rank: 0] Seed set to 42
INFO:lightning.fabric.utilities.seed:[rank: 0] Seed set to 42


catalog vocab size:4.99934 M


In [0]:
# catalog_id_index_temp, _ = get_index(index_path, "catalog_id", SPECIAl_TOKENS, False)
# index_to_catalog_merlin = dict(zip(catalog_id_index_temp.values(), catalog_id_index_temp.keys()))

In [0]:
index_to_catalog = dict(zip(catalog_id_index.values(), catalog_id_index.keys()))

In [0]:
index_to_sscat = dict(zip(sscat_id_index.values(), sscat_id_index.keys()))

# Model Loading

In [0]:
en_model_blob = bucket.get_blob(f'long_term_user_emb/checkpoint_dir/v15.1/epoch=2-step=60000-val_loss_5=0.00.ckpt')
en_model = en_model_blob.download_as_string()


# because model downloaded into string, need to convert it back
buffer = io.BytesIO(en_model)
state_dict = torch.load(buffer, map_location=torch.device('cpu'))

In [0]:
for key in list(state_dict['state_dict']):
    print(key, state_dict['state_dict'][key].shape)

pos_labels torch.Size([32768])
user_tower.future_mask_const torch.Size([1000, 1000])
user_tower.seq_diag_const torch.Size([1000, 1000])
user_tower.batch_indices torch.Size([512])
user_tower.event_type_embedding.weight torch.Size([5, 16])
user_tower.norm.weight torch.Size([16])
user_tower.norm.bias torch.Size([16])
user_tower.tf_encoder.layers.0.self_attn.in_proj_weight torch.Size([48, 16])
user_tower.tf_encoder.layers.0.self_attn.in_proj_bias torch.Size([48])
user_tower.tf_encoder.layers.0.self_attn.out_proj.weight torch.Size([16, 16])
user_tower.tf_encoder.layers.0.self_attn.out_proj.bias torch.Size([16])
user_tower.tf_encoder.layers.0.linear1.weight torch.Size([2048, 16])
user_tower.tf_encoder.layers.0.linear1.bias torch.Size([2048])
user_tower.tf_encoder.layers.0.linear2.weight torch.Size([16, 2048])
user_tower.tf_encoder.layers.0.linear2.bias torch.Size([16])
user_tower.tf_encoder.layers.0.norm1.weight torch.Size([16])
user_tower.tf_encoder.layers.0.norm1.bias torch.Size([16])
user

In [0]:
model_weights = state_dict["state_dict"]

In [0]:
final_weights = {}

# update keys by dropping `auto_encoder.`
for key in list(model_weights):
    if key not in ("cutoffs", "popular_labels", "pos_labels") and "catalog_tower" not in key and "catalog_embedding" not in key:
        final_weights[key.replace("user_tower.", "", 1)] = model_weights[key]

In [0]:
user_tower = UserTower(
    config["num_layers"],
    config["nhead"],
    config["behaviour_context_length"],
    config["embedding_dim"],
    event_type_vocab_size,
    sscat_id_vocab_size,
    config['batch_size']
)

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


In [0]:
user_tower.load_state_dict(final_weights)
user_tower = user_tower.eval()

In [0]:
final_weights_catalog = {}

# update keys by dropping `auto_encoder.`
for key in list(model_weights):
    if "catalog_tower" in key:
        final_weights_catalog[key.replace("catalog_tower.", "", 1)] = model_weights[key]

In [0]:
catalog_tower = CatalogTower(config["embedding_dim"])
# catalog_model = catalog_embedding(catalog_id_vocab_size,
#                 embedding_dimm=config["embedding_dim"]
#                  ).eval()
catalog_tower.load_state_dict(final_weights_catalog)
catalog_tower = catalog_tower.eval()

In [0]:
catalog_embedding = CatalogEmbedding(
    catalog_id_vocab_size=len(catalog_id_index),
    embedding_dim=16,
    use_pretrained=True,
    pretrained_weights=model_weights['catalog_embedding.catalog_embedding.weight']
)
catalog_embedding = catalog_embedding.eval()

# NNS Indexing

In [0]:
output = catalog_tower(torch.tensor([[[0,1,2,3]]]), catalog_embedding)
# output = torch.nn.functional.normalize(output, p=2, dim=-1)

In [0]:
import faiss

mlp_output = catalog_tower(torch.arange(catalog_id_vocab_size).reshape(1,1,catalog_id_vocab_size), catalog_embedding)[0][0]
mlp_output_arrays = mlp_output.detach().numpy()
index_v2 = faiss.IndexFlatIP(16)
faiss.normalize_L2(mlp_output_arrays)
index_v2.add(mlp_output_arrays)
print("done")

done


In [0]:
raw_embedding = catalog_embedding(torch.arange(catalog_id_vocab_size).reshape(1,1,catalog_id_vocab_size))[0][0]
raw_embedding_arrays = raw_embedding.detach().numpy()
index_raw = faiss.IndexFlatIP(16)
faiss.normalize_L2(raw_embedding_arrays)
index_raw.add(raw_embedding_arrays)

In [0]:
def get_nn(catalog_idx, index, mlp_output):
    return [int(index_to_catalog[x]) for x in index.search(np.array([mlp_output[catalog_idx]]), k=100)[1][0] if x not in [0,1]] 

In [0]:
display_images(get_nn(catalog_id_index[60541988], index_v2, mlp_output_arrays))

Women Jewellery 
 Bangles & Brace... 
 47 
 
 
 
 
 Women Ethnic We... 
 Blouses 
 13 
 
 
 
 
 Women Ethnic We... 
 Kurta Sets 
 37 
 
 
 
 
 Women Bags 
 Cross Body Bags... 
 11 
 
 
 
 
 Gifts 
 Personalized Gi... 
 47 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 2 
 
 
 
 
 Women Western W... 
 Tshirts 
 1 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 18 
 
 
 
 
 Kids - Girls We... 
 Tops & Tunics 
 5 
 
 
 
 
 Men Footwear 
 Shoe Accessorie... 
 12 
 
 
 
 
 Men Western Top... 
 Tshirts 
 4 
 
 
 
 
 Women Western W... 
 Dresses 
 4 
 
 
 
 
 Women Western W... 
 Tops & Tunics 
 8 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 6 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 7 
 
 
 
 
 Women Inner & S... 
 Briefs 
 2 
 
 
 
 
 Women Inner & S... 
 Nightsuits 
 3 
 
 
 
 
 Women Jewellery 
 Jewellery Set 
 48 
 
 
 
 
 Women Ethnic We... 
 Kurta Sets 
 5 
 
 
 
 
 Women Ethnic We... 
 Kurta Sets 
 41 
 
 
 
 
 Slipcovers(Sofa... 
 Chair Cover 
 10 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 1 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 34 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 15 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 2 
 
 
 
 
 Men Western Top... 
 Shirts 
 33 
 
 
 
 
 Women Jewellery 
 Jewellery Set 
 49 
 
 
 
 
 Clocks 
 Clocks 
 8 
 
 
 
 
 Slipcovers(Sofa... 
 Chair Cover 
 34 
 
 
 
 
 Women Western W... 
 Sweaters 
 40 
 
 
 
 
 Feeding & Water... 
 Feeding Bowls f... 
 6 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 7 
 
 
 
 
 Women Western W... 
 Jeans 
 29 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 1 
 
 
 
 
 Women Jewellery 
 Necklaces & Cha... 
 8 
 
 
 
 
 Women Jewellery 
 Bangles & Brace... 
 1 
 
 
 
 
 Women Western W... 
 Dresses 
 11 
 
 
 
 
 Kids - Boys Wes... 
 Oneseis & Rompe... 
 41 
 
 
 
 
 Women Accessori... 
 Hair extensions... 
 24 
 
 
 
 
 Women Jewellery 
 Jewellery Set 
 8 
 
 
 
 
 Nails Makeup 
 Nail Extensions 
 1 
 
 
 
 
 Pretend Play 
 Dress ups & cos... 
 13 
 
 
 
 
 Women Jewellery 
 Necklaces & Cha... 
 33 
 
 
 
 
 Women Ethnic We... 
 Dupatta Sets 
 7 
 
 
 
 
 Kids Accessorie... 
 Bathrobes 
 12 
 
 
 
 
 Women Ethnic We... 
 Kurtis & Kurtas 
 5 
 
 
 
 
 Women Ethnic We... 
 Ethnic Bottomwe... 
 13 
 
 
 
 
 Kids - Girls In... 
 Kurta Sets 
 9 
 
 
 
 
 Women Accessori... 
 Hair Bands 
 37 
 
 
 
 
 Fishing 
 Fishing Lures &... 
 3 
 
 
 
 
 Women Inner & S... 
 Shapewear 
 36 
 
 
 
 
 Tea 
 Black Tea 
 6 
 
 
 
 
 Soft Toys 
 Stuffed animals 
 48 
 
 
 
 
 Action & Toy Fi... 
 Toy Figures 
 2 
 
 
 
 
 Puja Accessorie... 
 Puja Articles 
 29 
 
 
 
 
 Clothing & Ward... 
 Clothes Hangers 
 21 
 
 
 
 
 Art & Craft Sup... 
 Yarns 
 24 
 
 
 
 
 Women Ethnic We... 
 Blouses 
 30 
 
 
 
 
 Kitchen & Dinin... 
 Kitchen Storage 
 36 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 27 
 
 
 
 
 Women Inner & S... 
 Thermal Tops 
 7 
 
 
 
 
 Women Ethnic We... 
 Suits & Dress M... 
 16 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 9 
 
 
 
 
 Maternity Care 
 Breast Creams 
 34 
 
 
 
 
 Women Western W... 
 Palazzos 
 1 
 
 
 
 
 Women Western W... 
 Jeans 
 21 
 
 
 
 
 Soft Toys 
 Teddy Bears 
 25 
 
 
 
 
 Kids - Girls We... 
 Frocks & Dresse... 
 6 
 
 
 
 
 Men Footwear 
 Crocs 
 5 
 
 
 
 
 Women Footwear 
 Flipflops & Sli... 
 33 
 
 
 
 
 Women Jewellery 
 Jewellery Set 
 16 
 
 
 
 
 Women Jewellery 
 Rings 
 41 
 
 
 
 
 Kids - Girls We... 
 Tops & Tunics 
 36 
 
 
 
 
 Men Accessories 
 Watchbands 
 6 
 
 
 
 
 Women Western W... 
 Gowns 
 29 
 
 
 
 
 Women Ethnic We... 
 Blouses 
 3 
 
 
 
 
 Women Ethnic We... 
 Dupatta Sets 
 2 
 
 
 
 
 Key Holders & W... 
 Key Holders 
 6 
 
 
 
 
 Women Western W... 
 Dresses 
 4 
 
 
 
 
 Clothing & Ward... 
 Clothes Covers 
 19 
 
 
 
 
 Women Western W... 
 Sweaters 
 27 
 
 
 
 
 Women Western W... 
 Palazzos 
 20 
 
 
 
 
 Women Ethnic We... 
 Kurtis & Kurtas 
 29 
 
 
 
 
 Baby & Toddler ... 
 Toddler Toys 
 25 
 
 
 
 
 Women Western W... 
 Dresses 
 14 
 
 
 
 
 Unisex Fragranc... 
 Attar 
 13 
 
 
 
 
 Men Western Top... 
 Shirts 
 46 
 
 
 
 
 Women Accessori... 
 A

In [0]:
display_images(get_nn(catalog_id_index[60541988], index_raw, raw_embedding_arrays))

Home Cleaning &... 
 Cleaning Suppli... 
 12 
 
 
 
 
 Women Jewellery 
 Jewellery Set 
 21 
 
 
 
 
 Women Jewellery 
 Bangles & Brace... 
 47 
 
 
 
 
 Women Ethnic We... 
 Kurtis & Kurtas 
 20 
 
 
 
 
 Women Ethnic We... 
 Dupatta Sets 
 41 
 
 
 
 
 Women Footwear 
 Casual Shoes 
 8 
 
 
 
 
 Women Jewellery 
 Bangles & Brace... 
 4 
 
 
 
 
 Women Accessori... 
 Hair Accessorie... 
 19 
 
 
 
 
 Women Jewellery 
 Bangles & Brace... 
 8 
 
 
 
 
 Snacks & Namkee... 
 Puffed Snacks 
 12 
 
 
 
 
 Women Jewellery 
 Jewellery Set 
 2 
 
 
 
 
 Kids - Girls We... 
 Frocks & Dresse... 
 1 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 3 
 
 
 
 
 Women Jewellery 
 Jewellery Set 
 50 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 7 
 
 
 
 
 Mobile & Access... 
 Mobile Chargers 
 26 
 
 
 
 
 Artificial Flor... 
 Artificial Plan... 
 20 
 
 
 
 
 Unisex Fragranc... 
 Unisex Perfumes 
 47 
 
 
 
 
 Women Western W... 
 Dresses 
 4 
 
 
 
 
 Clothing & Ward... 
 Clothes Covers 
 18 
 
 
 
 
 Women Jewellery 
 Anklets & Toe R... 
 14 
 
 
 
 
 Craft Kits 
 Sewing & Embroi... 
 15 
 
 
 
 
 Women Ethnic We... 
 Dupatta Sets 
 43 
 
 
 
 
 Face Care 
 Face Packs 
 28 
 
 
 
 
 Women Jewellery 
 Mangalsutras 
 50 
 
 
 
 
 Men Ethnic Wear 
 Kurtas 
 50 
 
 
 
 
 Men Ethnic Wear 
 Kurtas 
 46 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 15 
 
 
 
 
 Bath & Shower 
 Bath Combos and... 
 16 
 
 
 
 
 Face Care 
 Face Wash 
 50 
 
 
 
 
 Men Western Bot... 
 Jeans 
 4 
 
 
 
 
 Decorative Acce... 
 Wall Decor & Ha... 
 36 
 
 
 
 
 Women Ethnic We... 
 Dupatta Sets 
 7 
 
 
 
 
 Women Western W... 
 Capes, Shrugs &... 
 5 
 
 
 
 
 Women Footwear 
 Flipflops & Sli... 
 40 
 
 
 
 
 Photo Frames 
 Photo Frames 
 3 
 
 
 
 
 Women Ethnic We... 
 Dupattas 
 14 
 
 
 
 
 Women Western W... 
 Jumpsuits 
 19 
 
 
 
 
 Table Lamps 
 Table Lamps 
 48 
 
 
 
 
 School Supplies 
 Lunch Boxes & W... 
 7 
 
 
 
 
 Women Ethnic We... 
 Dupatta Sets 
 2 
 
 
 
 
 Women Western W... 
 Dresses 
 15 
 
 
 
 
 Women Western W... 
 Sweatshirts 
 18 
 
 
 
 
 Carpets & Rugs 
 Doormats 
 12 
 
 
 
 
 Men Accessories 
 Headbands 
 7 
 
 
 
 
 Women Western W... 
 Jackets 
 22 
 
 
 
 
 Women Ethnic We... 
 Kurtis & Kurtas 
 2 
 
 
 
 
 Women Western W... 
 Jeans 
 13 
 
 
 
 
 Women Ethnic We... 
 Kurtis & Kurtas 
 25 
 
 
 
 
 Headphones & Ea... 
 Wired Headphone... 
 5 
 
 
 
 
 Men Accessories 
 Analog Watches 
 2 
 
 
 
 
 Kitchen & Dinin... 
 Kitchen Storage 
 8 
 
 
 
 
 Kids - Girls We... 
 Clothing Sets 
 48 
 
 
 
 
 Women Jewellery 
 Rings 
 8 
 
 
 
 
 Women Jewellery 
 Earrings & Stud... 
 3 
 
 
 
 
 Accupressure 
 Accupressure su... 
 5 
 
 
 
 
 Men Footwear 
 Mojaris and Jut... 
 8 
 
 
 
 
 Soft Toys 
 Stuffed animals 
 48 
 
 
 
 
 Women Ethnic We... 
 Ready To Wear S... 
 7 
 
 
 
 
 Women Bags 
 Backpacks 
 6 
 
 
 
 
 Men Accessories 
 Ties 
 26 
 
 
 
 
 Maternity Care 
 Breast Creams 
 34 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 7 
 
 
 
 
 Women Jewellery 
 Pendants & Lock... 
 4 
 
 
 
 
 Face Makeup 
 Blush 
 39 
 
 
 
 
 Cases & Covers 
 Mobile Cases & ... 
 9 
 
 
 
 
 Cleaning Tools 
 Brushes 
 48 
 
 
 
 
 Women Ethnic We... 
 Dupatta Sets 
 17 
 
 
 
 
 Audio & Video A... 
 Car Bluetooth K... 
 3 
 
 
 
 
 Women Jewellery 
 Jewellery Set 
 16 
 
 
 
 
 Kid's Toys 
 Others 
 3 
 
 
 
 
 Women Western W... 
 Gowns 
 29 
 
 
 
 
 Women Inner & S... 
 Nightdress 
 10 
 
 
 
 
 Gifts 
 Gifting Mugs 
 1 
 
 
 
 
 Hair Styling & ... 
 Hair Straighten... 
 10 
 
 
 
 
 Women Jewellery 
 Jewellery Set 
 5 
 
 
 
 
 Men Western Top... 
 Shirts 
 1 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 22 
 
 
 
 
 Men Western Top... 
 Shirts 
 33 
 
 
 
 
 Clothing & Ward... 
 Clothes Covers 
 41 
 
 
 
 
 Women Ethnic We... 
 Kurta Sets 
 1 
 
 
 
 
 Kids - Girls We... 
 Sweatshirts & H... 
 26 
 
 
 
 
 Women Jewellery 
 Jewellery Set 
 11 
 
 
 
 
 Pretend Play 
 Dress ups & cos... 
 40 
 
 
 
 
 Women Ethnic We... 
 Sarees 
 11 
 
 
 
 
 Men Footwear 
 Sports Shoes 
 4

# User Qualitative Analysis

In [0]:
from pyspark.sql import functions as F

In [0]:
taxonomy = spark.sql("""select distinct sscat_id, sscat 
                    from gold.product_info 
                    where catalog_id is not null""")

pdf_taxonomy = taxonomy.toPandas()


In [0]:
df = spark.read.parquet(f"gs://gcs-dsci-fryou-fy-dev-prd/{config['filtered_journeys_path']}").limit(100).collect()

In [0]:
import torch.nn.functional as F

i = 10
interaction_list = [ele.asDict() for ele in df[i]['interaction_list']]
# len(interaction_list)
result = get_recos(interaction_list, 10)

9 208 217


In [0]:
def calculate_intersection_ratio(input_set, predicted_set):
    intersection = input_set.intersection(predicted_set)
    overlap = len(intersection) / len(predicted_set) if predicted_set else 0
    return overlap


for i in range(10):
    interaction_list = [ele.asDict() for ele in df[i]['interaction_list']]
    X, sscat, sscat_logits = get_recos(interaction_list, 5)

    input_sscat = set(X['sscat_id_seq'])
    predicted_sscat = set(sscat)
    actual_sscat = set(index_to_sscat[ele] for ele in X['positive_sscat_id_seq'].tolist()[0])

        
    display(f"Top {percentile}%:")
    display(calculate_intersection_ratio(input_sscat, predicted_sscat))
    display(calculate_intersection_ratio(actual_sscat, predicted_sscat))
    display(calculate_intersection_ratio(input_sscat, actual_sscat))

8 143 151


'Top <function percentile at 0x7a14998793f0>%:'

0.0

0.2

0.0

...

26 432 458


'Top <function percentile at 0x7a14998793f0>%:'

0.0

0.6

0.0

In [0]:
# def calculate_intersection_ratio_ap(actual_set, predicted_set):
#     intersection = actual_set.intersection(predicted_set)
#     overlap = len(intersection) / len(predicted_set) if predicted_set else 0
#     return overlap

In [0]:
def calculate_overlap(predicted_sscat, actual_sscat):
    intersection = predicted_sscat.intersection(actual_sscat)
    overlap_ratio = len(intersection) / len(actual_sscat) if actual_sscat else 0
    display(pdf_taxonomy[pdf_taxonomy.sscat_id.isin(intersection)][['sscat_id', "sscat"]].style.set_caption("Intersection of Predicted and Actual SSCAT"))
    display("----------")
    display(pdf_taxonomy[pdf_taxonomy.sscat_id.isin(predicted_sscat)][['sscat_id', "sscat"]].style.set_caption("Predicted SSCAT"))
    display("----------")
    display(pdf_taxonomy[pdf_taxonomy.sscat_id.isin(actual_sscat)][['sscat_id', "sscat"]].style.set_caption("Actual SSCAT"))

    return overlap_ratio

In [0]:
sscat_logits.shape

torch.Size([1, 1, 3606])

In [0]:
print(sscat_logits[0].topk(5))

torch.return_types.topk(
values=tensor([[0.6159, 0.5855, 0.5489, 0.5467, 0.5093]], grad_fn=<TopkBackward0>),
indices=tensor([[3088,  949,  736, 1683,  221]]))


In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, IntegerType

def filter_actual_sscat(sscat_ids):
    excluded_ids = {1004, 1001, 1853, 1003, 1093, 1020}
    intersection = set(sscat_ids).intersection(excluded_ids)
    return len(intersection)

for i in range(100):
    interaction_list = [ele.asDict() for ele in df[i]['interaction_list']]
    X, sscat, sscat_logits = get_recos(interaction_list, 5)
    print(sscat_logits[0].topk(5))
    # print(X)
    input_sscat = set(index_to_sscat[ele] for ele in X['sscat_id_seq'].tolist()[0])
    actual_sscat = set(index_to_sscat[ele] for ele in X['positive_sscat_id_seq'].tolist()[0])

    # predicted_sscat = set(sscat)
    overlap = filter_actual_sscat(list(input_sscat))
    if overlap == 0:
        display(pdf_taxonomy[pdf_taxonomy.sscat_id.isin(predicted_sscat)][['sscat_id', "sscat"]].style.set_caption("Predicted SSCAT"))
        display("----------")
        display(pdf_taxonomy[pdf_taxonomy.sscat_id.isin(input_sscat)][['sscat_id', "sscat"]].style.set_caption("input SSCAT"))
        display(pdf_taxonomy[pdf_taxonomy.sscat_id.isin(actual_sscat)][['sscat_id', "sscat"]].style.set_caption("Actual SSCAT"))


8 143 151
torch.return_types.topk(
values=tensor([[0.5845, 0.5692, 0.5321, 0.5137, 0.4996]], grad_fn=<TopkBackward0>),
indices=tensor([[3088,  949,  736, 1683, 1468]]))
5 213 218
torch.return_types.topk(
values=tensor([[0.6016, 0.5792, 0.5430, 0.5337, 0.5050]], grad_fn=<TopkBackward0>),
indices=tensor([[3088,  949,  736, 1683, 1468]]))
30 144 174
torch.return_types.topk(
values=tensor([[0.5994, 0.5772, 0.5398, 0.5286, 0.5033]], grad_fn=<TopkBackward0>),
indices=tensor([[3088,  949,  736, 1683, 1468]]))
27 192 219
torch.return_types.topk(
values=tensor([[0.6151, 0.5848, 0.5486, 0.5447, 0.5093]], grad_fn=<TopkBackward0>),
indices=tensor([[3088,  949,  736, 1683,  221]]))
7 356 363
torch.return_types.topk(
values=tensor([[0.6323, 0.5921, 0.5642, 0.5567, 0.5156]], grad_fn=<TopkBackward0>),
indices=tensor([[3088,  949, 1683,  736,  221]]))
125 262 387
torch.return_types.topk(
values=tensor([[0.6261, 0.5901, 0.5557, 0.5533, 0.5135]], grad_fn=<TopkBackward0>),
indices=tensor([[3088,  949, 168

,sscat_id,sscat
506,1020,Tops & Tunics
736,1001,Kurtis & Kurtas
2365,1003,Kurta Sets
3116,1004,Sarees
3166,1853,Dupatta Sets


'----------'

,sscat_id,sscat
69,5543,Hair Care Combo
74,4022,Embroidery Threads
458,1175,Shorts & Capris
478,1239,Flip Flops
613,1035,Leggings & Tights
640,5534,Shampoo
684,1238,Sandals
712,1025,Dresses
856,1206,Shirts
915,1167,Sandals


,sscat_id,sscat
2192,1070,Flipflops & Slippers


...

,sscat_id,sscat
506,1020,Tops & Tunics
736,1001,Kurtis & Kurtas
2365,1003,Kurta Sets
3116,1004,Sarees
3166,1853,Dupatta Sets


'----------'

,sscat_id,sscat
346,2709,Dashboard Figurines & Idols
478,1239,Flip Flops
481,1226,Sunglasses
676,3715,Walnuts
1249,5486,Makeup Kits
1252,1402,Tracksuits
1451,3624,Pumpkin Seeds
1776,1915,Omega Supplements
1960,5626,Face Oil & Serums
2010,1943,Nightsuits


,sscat_id,sscat
506,1020,Tops & Tunics
737,1034,Trousers & Pants
856,1206,Shirts
1500,1081,Belts
1869,1056,Thermal Tops
1987,2152,Analog Watches
3817,1207,Sweatshirts


1 135 136
torch.return_types.topk(
values=tensor([[0.5715, 0.5614, 0.5266, 0.5020, 0.4988]], grad_fn=<TopkBackward0>),
indices=tensor([[3088,  949,  736, 1683, 1799]]))
12 100 112
torch.return_types.topk(
values=tensor([[0.5794, 0.5653, 0.5287, 0.5074, 0.4972]], grad_fn=<TopkBackward0>),
indices=tensor([[3088,  949,  736, 1683, 1799]]))
49 116 165
torch.return_types.topk(
values=tensor([[0.5909, 0.5733, 0.5359, 0.5206, 0.5019]], grad_fn=<TopkBackward0>),
indices=tensor([[3088,  949,  736, 1683, 1468]]))
26 300 326
torch.return_types.topk(
values=tensor([[0.6158, 0.5860, 0.5503, 0.5489, 0.5077]], grad_fn=<TopkBackward0>),
indices=tensor([[3088,  949,  736, 1683,  221]]))
14 113 127
torch.return_types.topk(
values=tensor([[0.5784, 0.5644, 0.5312, 0.5107, 0.4998]], grad_fn=<TopkBackward0>),
indices=tensor([[3088,  949,  736, 1683, 1799]]))
10 174 184
torch.return_types.topk(
values=tensor([[0.5906, 0.5723, 0.5359, 0.5204, 0.5015]], grad_fn=<TopkBackward0>),
indices=tensor([[3088,  949,  7

In [0]:
for i in range(10):
    interaction_list = [ele.asDict() for ele in df[i]['interaction_list']]
    X, sscat, sscat_logits = get_recos(interaction_list, 5)
    # print(sscat_logits)
    actual_sscat = set(index_to_sscat[ele] for ele in X['positive_sscat_id_seq'].tolist()[0])
    predicted_sscat = set(sscat)
    overlap = calculate_overlap(predicted_sscat, actual_sscat)
    print(f"Overlap ratio for iteration {i}: {overlap}")

8 143 151


,sscat_id,sscat
506,1020,Tops & Tunics


'----------'

,sscat_id,sscat
506,1020,Tops & Tunics
736,1001,Kurtis & Kurtas
2365,1003,Kurta Sets
3116,1004,Sarees
3166,1853,Dupatta Sets


'----------'

...

'----------'

,sscat_id,sscat
506,1020,Tops & Tunics
736,1001,Kurtis & Kurtas
2365,1003,Kurta Sets
3116,1004,Sarees
3166,1853,Dupatta Sets


'----------'

,sscat_id,sscat
21,1032,Jeans
506,1020,Tops & Tunics
2365,1003,Kurta Sets
3116,1004,Sarees


Overlap ratio for iteration 9: 0.75


# Overlaps between input and future window

In [0]:
output_path = 'gs://gcs-dsci-fryou-fy-dev-prd/long_term_user_emb/'
version = "v15"
data = spark.read.parquet(f"{output_path}/user_seq_final/{version}/")
interaction_list = data.select("interaction_list")

In [0]:
from pyspark.sql.functions import col, udf, explode, count, lit, max as spark_max
from pyspark.sql.types import FloatType
from datetime import timedelta, datetime

def get_overlap_sscat_udf(row):
    end_time = row[-1]['event_time']
    start_time = end_time - timedelta(days=14)
    input_seq = [event['sscat_id'] for event in row if event['event_time'] < start_time]
    positive_seq = [event['sscat_id'] for event in row if event['event_time'] >= start_time]
    input_sscat_set = set(input_seq)
    positive_sscat_set = set(positive_seq)
    overlap_sscat = input_sscat_set.intersection(positive_sscat_set)
    overlap_ratio = len(overlap_sscat) / len(positive_sscat_set) if positive_seq else 0
    return overlap_ratio

get_overlap_sscat = udf(get_overlap_sscat_udf, FloatType())

sdf_with_overlap = data.withColumn("overlap_ratio", get_overlap_sscat(col("interaction_list")))

percentiles = sdf_with_overlap.approxQuantile("overlap_ratio", [0.5], 0.0)
percentiles

[0.5]

In [0]:
def get_sscat_count_udf(row):
    end_time = row[-1]['event_time']
    start_time = end_time - timedelta(days=14)
    # input_seq = [event['sscat_id'] for event in row if event['event_time'] >= start_time]
    positive_seq = [event['sscat_id'] for event in row if event['event_time'] >= start_time]
    # input_sscat_set = set(input_seq)
    positive_sscat_set = set(positive_seq)
    return len(positive_sscat_set)

get_overlap_sscat = udf(get_sscat_count_udf, IntegerType())

sdf_with_overlap = data.withColumn("overlap_ratio", get_overlap_sscat(col("interaction_list")))

# sdf_with_overlap.display()

percentiles = sdf_with_overlap.approxQuantile("overlap_ratio", [0.5], 0.0)
percentiles

[5.0]

In [0]:
from pyspark.sql.functions import col, count, lit, max as spark_max
from datetime import timedelta


def get_sscat_futures_udf(row):
    end_time = row[-1]['event_time']
    start_time = end_time - timedelta(days=14)
    positive_seq = [event['sscat_id'] for event in row if event['event_time'] >= start_time]
    # positive_sscat_set = set(positive_seq)
    return positive_seq

get_sscat_futures = udf(get_sscat_futures_udf, ArrayType(IntegerType()))

sscat_futures = data.withColumn("future_sscats", get_sscat_futures(col("interaction_list")))

sscat_futures = sscat_futures.selectExpr("explode(future_sscats) as sscat_id").groupBy("sscat_id").agg(count(lit(1)))

sscat_futures.display()

sscat_id,count(1)
1886,28235
5919,60846
4455,3800
1047,253576
5906,187482
1181,489520
1622,5301
3431,5470
3635,717
3576,1407


In [0]:
from pyspark.sql.functions import col, udf, expr
from pyspark.sql.types import FloatType
from datetime import timedelta

def get_overlap_scat_udf(row):
    end_time = row[-1]['event_time']
    start_time = end_time - timedelta(days=14)
    input_seq = [event['scat_id'] for event in row if event['event_time'] < start_time]
    positive_seq = [event['scat_id'] for event in row if event['event_time'] >= start_time]
    # display(input_seq, positive_seq)
    input_scat_set = set(input_seq)
    positive_scat_set = set(positive_seq)
    overlap_scat = input_scat_set.intersection(positive_scat_set)
    overlap_ratio = len(overlap_scat) / len(positive_scat_set) if positive_seq else 0
    return overlap_ratio

get_overlap_scat = udf(get_overlap_scat_udf, FloatType())

sdf_with_overlap = data.withColumn("overlap_ratio", get_overlap_scat(col("interaction_list")))
# sdf_with_overlap.display()

percentiles = sdf_with_overlap.approxQuantile("overlap_ratio", [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 0.01)
percentiles

[0.25,
 0.47058823704719543,
 0.5,
 0.6666666865348816,
 0.7142857313156128,
 0.800000011920929,
 1.0,
 1.0,
 1.0,
 1.0]

# Insights
- positive sscat seq length - 5
- input seq length med - 190
- median sscat overlap - 0.5
- top 5 sscats - 30% of positive
- overlap of input and future ~ 71%


# Extra

In [0]:
pdf_taxonomy[pdf_taxonomy.sscat_id.isin(actual_sscat)][['sscat_id', "sscat"]]

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecution(ChauffeurState.scala:1315)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:1032)
	at com.databricks.logging.Usage

In [0]:
pdf_taxonomy[pdf_taxonomy.sscat_id.isin(predicted_sscat)]

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecution(ChauffeurState.scala:1315)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:1032)
	at com.databricks.logging.Usage

In [0]:
a = np.random.random_sample(10)

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecution(ChauffeurState.scala:1315)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:1032)
	at com.databricks.logging.Usage

In [0]:
a[::-1]*0.01

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecution(ChauffeurState.scala:1315)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:1032)
	at com.databricks.logging.Usage

In [0]:
import torch.nn as nn

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecution(ChauffeurState.scala:1315)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:1032)
	at com.databricks.logging.Usage

In [0]:
nn.functional.softmax(torch.tensor(a*0.01), dim=0).sort(descending=True)[0]

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecution(ChauffeurState.scala:1315)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:1032)
	at com.databricks.logging.Usage

In [0]:
nn.functional.softmax(torch.tensor(a), dim=0).sort(descending=True)[0]

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecution(ChauffeurState.scala:1315)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:1032)
	at com.databricks.logging.Usage